# Codificación de valores faltantes

In [21]:
import pandas as pd
import numpy as np
import janitor

In [6]:
%run pandas-missing-extension.ipynb

<div class="alert alert-warning" role="alert" style="background-color: #FFF3CD; border: 1px solid #D4CA9E; border-radius: 5px;">
    <b style="font-size: 1.5em; color: #856404;">🚧 Advertencia</b>
    <p>
        Al igual que cada persona es una nueva puerta a un mundo diferente, los <b>valores faltantes</b> existen en diferentes formas y colores. Al trabajar con valores faltantes será crítico entender sus distintas representaciones. A pesar de que el conjunto de datos de trabajo pareciera que no contiene valores faltantes, deberás ser capaz de ir más allá de lo observado a simple vista para remover el manto tras el cual se esconde lo desconocido.
    </p>
</div>

## Valores comúnmente asociados a valores faltantes


### Cadenas de texto

In [1]:
common_na_strings = (
    "missing",
    "NA",
    "N A",
    "N/A",
    "#N/A",
    "NA ",
    " NA",
    "N /A",
    "N / A",
    " N / A",
    "N / A ",
    "na",
    "n a",
    "n/a",
    "na ",
    " na",
    "n /a",
    "n / a",
    " a / a",
    "n / a ",
    "NULL",
    "null",
    "",
    "?",
    "*",
    ".",
    " ",
)

### Números

In [2]:
common_na_numbers = (-9, -99, -999, -9999, 9999, 66, 77, 88, -1)

## ¿Cómo encontrar los valores comúnmente asociados a valores faltantes?

In [5]:
missing_data_example_df = pd.DataFrame.from_dict(
    dict(
        x = [1, 3, "NA", -99, -98, -99],
        y = ["A", "N/A", "NA", "E", "F", "G"],
        z = [-100, -99, -98, -101, -1, -1]
    )
)

missing_data_example_df

,x,y,z
0,1,A,-100
1,3,N/A,-99
2,NA,NA,-98
3,-99,E,-101
4,-98,F,-1
5,-99,G,-1


In [7]:
missing_data_example_df.missing.number_missing()

0

### Revisar tipos de datos

In [8]:
missing_data_example_df.dtypes

x    object
y    object
z     int64
dtype: object

### Revisar los valores únicos de los datos

In [9]:
missing_data_example_df.x.unique()

array([1, 3, 'NA', -99, -98], dtype=object)

In [10]:
missing_data_example_df.select_dtypes(object).apply(pd.unique)

x     [1, 3, NA, -99, -98]
y    [A, N/A, NA, E, F, G]
dtype: object

## Sustiyuyendo valores comúnmente asociados a valores faltantes

### Sustitución global

In [13]:
missing_data_example_df.replace([-99, "NA"], value=np.nan)

,x,y,z
0,1.0,A,-100.0
1,3.0,N/A,NaN
2,NaN,NaN,-98.0
3,NaN,E,-101.0
4,-98.0,F,-1.0
5,NaN,G,-1.0


### Sustitución dirigida

In [14]:
missing_data_example_df.replace(to_replace={"x":{-99:np.nan}})

,x,y,z
0,1,A,-100
1,3,N/A,-99
2,NA,NA,-98
3,NaN,E,-101
4,-98,F,-1
5,NaN,G,-1


# Conversion de valores faltantes explicitos e implicitos

In [15]:
implicit_to_explicit_df = pd.DataFrame.from_dict(
    data={
        "name": ["lynn", "lynn", "lynn", "zelda"],
        "time": ["morning", "afternoon", "night", "morning"],
        "value": [350, 310, np.nan, 320]
    }
)

implicit_to_explicit_df

,name,time,value
0,lynn,morning,350.0
1,lynn,afternoon,310.0
2,lynn,night,NaN
3,zelda,morning,320.0


## Pivotar tabla de datos

Estrategia para la identificacion de valores faltantes implicitos

Pivotar la tabla de datos. Pivotar es tomar una columna de referencia sobre la cual se quiere extender sus categorias de manera que muestre una nueva visualizacion de los datos

In [19]:
explicit_df = implicit_to_explicit_df.pivot(index="name", columns="time", values="value")
explicit_df

time,afternoon,morning,night
name,,,
lynn,310.0,350.0,NaN
zelda,NaN,320.0,NaN


## Cuantificar ocurrencia de n - tuplas

In [20]:
implicit_to_explicit_df.value_counts(
    subset=["name"]
).reset_index(name="n").query("n<3")

,name,n
1,zelda,1


# Exponer filas faltantes implícitas a explícitas


<div class="alert alert-info" role="alert">
    <b style="font-size: 1.5em;">ℹ️ Información</b>
    <p>
La librería de Janitor es una librería de Python que proporciona un conjunto de funciones útiles para limpiar y preparar datos en Pandas. Esta biblioteca está diseñada para ayudar en la preparación de datos para el análisis, lo que puede ahorrar tiempo y reducir errores. Algunas de las funciones más utilizadas de Janitor en Pandas incluyen.

- **clean_names():** esta función se utiliza para limpiar y estandarizar los nombres de las columnas en un DataFrame. Elimina espacios, caracteres especiales y convierte los nombres de las columnas a minúsculas.

- **remove_empty() y remove_na():** estas funciones se utilizan para eliminar filas y columnas que contengan valores faltantes o nulos.

- **coalesce():** esta función se utiliza para reemplazar los valores nulos o faltantes en una columna con otro valor especificado.

- **expand_column():** esta función se utiliza para dividir una columna en múltiples columnas utilizando un separador específico.

- **get_dupes():** esta función se utiliza para identificar filas duplicadas en un DataFrame.


- **groupby_summarize():** esta función se utiliza para agrupar los datos en un DataFrame y calcular estadísticas resumidas, como la media, la suma o la desviación estándar.

- **pivot_longer() y pivot_wider():** estas funciones se utilizan para transformar un DataFrame de formato ancho a formato largo o viceversa.
En resumen, la librería de Janitor es una herramienta útil para preparar y limpiar datos en Pandas. Sus funciones son útiles para estandarizar nombres de columnas, eliminar filas y columnas con valores faltantes, identificar filas duplicadas y realizar transformaciones de formato en los datos.

    </p>
</div>

## Exponer n-tulas de valores faltantes

### Ejemplo, encontrar los pares faltates de name y time

In [22]:
(
    implicit_to_explicit_df
    #janitor
    .complete(
        "name",
        "time"
    )
    
    )

,name,time,value
0,lynn,morning,350.0
1,lynn,afternoon,310.0
2,lynn,night,NaN
3,zelda,morning,320.0
4,zelda,afternoon,NaN
5,zelda,night,NaN


## limitar la exposición de n-tuplas de valores faltantes

In [23]:
(
    implicit_to_explicit_df
    #janitor
    .complete(
        { "name": ["lyn","zelda"]},
        {"time":["morning","afternoon"]},
        sort=True
    )
    
    )

,name,time,value
0,lyn,afternoon,NaN
1,lyn,morning,NaN
2,zelda,afternoon,NaN
3,zelda,morning,320.0
4,lynn,morning,350.0
5,lynn,afternoon,310.0
6,lynn,night,NaN


## Rellenar los valores faltantes

In [26]:
(
    implicit_to_explicit_df
    .complete(
        "name",
        "time",
        fill_value=np.nan
    )
)

/Users/Sebastian/Desktop/Cursos Platzi/Datos faltantes/env_data/lib/python3.9/site-packages/janitor/functions/complete.py:308: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  out = out.fillna(fill_value, downcast="infer")


,name,time,value
0,lynn,morning,350.0
1,lynn,afternoon,310.0
2,lynn,night,NaN
3,zelda,morning,320.0
4,zelda,afternoon,NaN
5,zelda,night,NaN


## Limitar el rellenado de valores faltantes implícitos

In [27]:
# si ya faltaba ponen nan, si no faltba pone 0 es una chimba
(
    implicit_to_explicit_df
    .complete(
        "name",
        "time",
        fill_value=0,
        explicit=False
    )
)

,name,time,value
0,lynn,morning,350.0
1,lynn,afternoon,310.0
2,lynn,night,NaN
3,zelda,morning,320.0
4,zelda,afternoon,0.0
5,zelda,night,0.0
